In [ ]:
!pip install subword-nmt evaluate

In [1]:
from tqdm import tqdm

In [2]:
import pandas as pd

In [1]:
from oml.datasets.base import DatasetWithLabels
from oml.losses.triplet import TripletLossWithMiner
from oml.miners.inbatch_hard_tri import HardTripletsMiner
from oml.models import ViTExtractor
from oml.samplers.balance import BalanceSampler
from oml.utils.download_mock_dataset import download_mock_dataset

from IPython.display import clear_output
from tqdm import tqdm, trange
import numpy as np
import pandas as pd
from tqdm import tqdm

from nltk.tokenize import WordPunctTokenizer
from subword_nmt.learn_bpe import learn_bpe
from subword_nmt.apply_bpe import BPE
from nltk.translate.bleu_score import corpus_bleu
from sklearn.model_selection import train_test_split
from vocab import Vocab
from sklearn.metrics import roc_auc_score

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import ExponentialLR
from transformers import (DataCollatorForSeq2Seq, AutoTokenizer,
                          AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer)

import datasets
import evaluate

import matplotlib.pyplot as plt
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
df1 = pd.read_csv('./datasets/dataset_1.csv').drop('Unnamed: 0', axis=1)
df2 = pd.read_csv('./datasets/dataset_2.csv').drop('Unnamed: 0', axis=1)
df3 = pd.read_csv('./datasets/dataset_3.csv').drop('Unnamed: 0', axis=1)
df4 = pd.read_csv('./datasets/dataset_4.csv').drop('Unnamed: 0', axis=1)
# df5 = pd.read_csv('./datasets/dataset_5.csv').drop('Unnamed: 0', axis=1)

df = pd.concat((df1, df2, df3, df4)).dropna()

In [4]:
building = pd.read_csv('./additional_data/building_20230808.csv')
area = pd.read_csv('./additional_data/area_20230808.csv')
areatype = pd.read_csv('./additional_data/areatype_20230808.csv')
district = pd.read_csv('./additional_data/district_20230808.csv')
geonim = pd.read_csv('./additional_data/geonim_20230808.csv')
geonimtype = pd.read_csv('./additional_data/geonimtype_20230808.csv')
prefix = pd.read_csv('./additional_data/prefix_20230808.csv')
subrf = pd.read_csv('./additional_data/subrf_20230808.csv')
town = pd.read_csv('./additional_data/town_20230808.csv')

<ipython-input-4-dfe38b4e1cb0>:1: DtypeWarning: Columns (4,6,7,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  building = pd.read_csv('./additional_data/building_20230808.csv')


In [8]:
building = building[building['is_actual']]

In [9]:
df = pd.concat((building[['id', 'full_address']].rename(columns={'full_address': 'address', 'id': 'target_building_id'}),
                building[['id', 'short_address']].rename(columns={'short_address': 'address', 'id': 'target_building_id'}),
                df[['address', 'target_building_id']],
                df[['target_address', 'target_building_id']].rename(columns={'target_address': 'address'})
               ))

In [11]:
df.to_csv('train.csv')

In [134]:
df4[df4['address'] == 'Фонтанки наб., 203БВ']['target_address'][9888]

'г.Санкт-Петербург, набережная реки Фонтанки, дом 203, литера БВ'

In [123]:
test.to_csv('test.csv')

In [122]:
train.to_csv('train.csv')

In [108]:
building[['id', 'short_address']]

,id,prefix_id,district_id,house,corpus,liter,villa,parcel,full_address,is_updated,is_actual,type,municipality_id,short_address,post_prefix,build_number
0,56343,11132,35,12,NaN,А,NaN,NaN,"город Пушкин, Кедринская улица, дом 12",True,False,NaN,107.0,"г.Пушкин, Кедринская ул., д. 12",NaN,NaN
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,"поселок Ушково, Пляжевая улица, дом 4Б",True,False,NaN,128.0,"пос. Ушково, Пляжевая ул., д. 4Б",NaN,NaN
2,7134,6469,15,30,2,Е,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова, до...",True,False,Нежилое,30.0,"г.Санкт-Петербург, пр. Маршала Жукова, д. 30, ...",198303,NaN
3,124415,7838,38,5,2,А,NaN,NaN,"поселок Белоостров, Дюны, Центральная улица, д...",True,False,Нежилое,110.0,"пос. Белоостров, Дюны, Центральная ул., д. 5, ...",NaN,NaN
4,185368,4224,38,28,NaN,Б,NaN,NaN,"поселок Песочный, Речная улица, дом 28, литера Б",True,False,NaN,118.0,"пос. Песочный, Речная ул., д. 28, л. Б",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166640,18941,5289,32,3,NaN,В,NaN,NaN,"г.Санкт-Петербург, Бородинская улица, дом 3, л...",True,True,Нежилое,4.0,"г.Санкт-Петербург, Бородинская ул., д. 3, л. В",190000.0,NaN
166641,18942,5031,35,34/3,NaN,А,NaN,NaN,"посёлок Александровская, 2-я линия, дом 34/3, ...",True,True,Жилое,109.0,"пос. Александровская, 2-я линия, д. 34/3, л. А",196631.0,NaN
166642,18943,5623,38,39,NaN,NaN,NaN,NaN,"посёлок Ушково, Детская улица, дом 39",True,False,NaN,128.0,"пос. Ушково, Детская ул., д. 39",NaN,NaN
166643,18944,5787,34,25,NaN,Б,NaN,NaN,"г.Санкт-Петербург, Заусадебная улица, дом 25, ...",True,False,Нежилое,68.0,"г.Санкт-Петербург, Заусадебная ул., д. 25, л. Б",NaN,NaN


In [87]:
building[building['full_address'] == 'г.Санкт-Петербург, проспект Косыгина, дом 9, корпус 3, литера А']

,id,prefix_id,district_id,house,corpus,liter,villa,parcel,full_address,is_updated,is_actual,type,municipality_id,short_address,post_prefix,build_number
57721,54298,6087,11,9,3,А,NaN,NaN,"г.Санкт-Петербург, проспект Косыгина, дом 9, к...",True,True,Нежилое,35.0,"г.Санкт-Петербург, пр. Косыгина, д. 9, к. 3, л. А",195298.0,NaN


In [88]:
building

,id,prefix_id,district_id,house,corpus,liter,villa,parcel,full_address,is_updated,is_actual,type,municipality_id,short_address,post_prefix,build_number
0,56343,11132,35,12,NaN,А,NaN,NaN,"город Пушкин, Кедринская улица, дом 12",True,False,NaN,107.0,"г.Пушкин, Кедринская ул., д. 12",NaN,NaN
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,"поселок Ушково, Пляжевая улица, дом 4Б",True,False,NaN,128.0,"пос. Ушково, Пляжевая ул., д. 4Б",NaN,NaN
2,7134,6469,15,30,2,Е,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова, до...",True,False,Нежилое,30.0,"г.Санкт-Петербург, пр. Маршала Жукова, д. 30, ...",198303,NaN
3,124415,7838,38,5,2,А,NaN,NaN,"поселок Белоостров, Дюны, Центральная улица, д...",True,False,Нежилое,110.0,"пос. Белоостров, Дюны, Центральная ул., д. 5, ...",NaN,NaN
4,185368,4224,38,28,NaN,Б,NaN,NaN,"поселок Песочный, Речная улица, дом 28, литера Б",True,False,NaN,118.0,"пос. Песочный, Речная ул., д. 28, л. Б",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166640,18941,5289,32,3,NaN,В,NaN,NaN,"г.Санкт-Петербург, Бородинская улица, дом 3, л...",True,True,Нежилое,4.0,"г.Санкт-Петербург, Бородинская ул., д. 3, л. В",190000.0,NaN
166641,18942,5031,35,34/3,NaN,А,NaN,NaN,"посёлок Александровская, 2-я линия, дом 34/3, ...",True,True,Жилое,109.0,"пос. Александровская, 2-я линия, д. 34/3, л. А",196631.0,NaN
166642,18943,5623,38,39,NaN,NaN,NaN,NaN,"посёлок Ушково, Детская улица, дом 39",True,False,NaN,128.0,"пос. Ушково, Детская ул., д. 39",NaN,NaN
166643,18944,5787,34,25,NaN,Б,NaN,NaN,"г.Санкт-Петербург, Заусадебная улица, дом 25, ...",True,False,Нежилое,68.0,"г.Санкт-Петербург, Заусадебная ул., д. 25, л. Б",NaN,NaN


In [86]:
df[df['address'] == 'г.Санкт-Петербург, проспект Косыгина, дом 9, корпус 3, литера А']

,address,target_building_id
914,"г.Санкт-Петербург, проспект Косыгина, дом 9, к...",54298.0


In [103]:
#Импортируем необходимые модули из Natasha и объявляем переменные
from natasha import (
    Segmenter,
    MorphVocab,
    LOC,
    AddrExtractor    
)
segmenter = Segmenter()
morph_vocab = MorphVocab()
addr_extractor = AddrExtractor(morph_vocab)
#Текст, содержащий адреса
text = "текст текст текст Россия, Воронежская область, г. Воронеж, район Ленинский, ул. 9 Января, д. 68к текст текст текст"
text = 'текст текст текст г.Санкт-Петербург, проспект Косыгина, дом 9, корпус 3, литера А текст текст текст '
text = 'askfhaghoiewhg поселок Ушково, Пляжевая улица, дом 4Б ihaoidfh qwoih'
#Извлечение адреса из текста
matches = addr_extractor(text)
facts = [i.fact.as_json for i in matches]
#Цикл для вывода адреса в удобной форме
for i in range(len(facts)):
    tmp = list(facts[i].values())
    print(tmp[1], tmp[0])

посёлок Ушково
улица Пляжевая
дом 4Б


In [73]:
building

,id,prefix_id,district_id,house,corpus,liter,villa,parcel,full_address,is_updated,is_actual,type,municipality_id,short_address,post_prefix,build_number
0,56343,11132,35,12,NaN,А,NaN,NaN,"город Пушкин, Кедринская улица, дом 12",True,False,NaN,107.0,"г.Пушкин, Кедринская ул., д. 12",NaN,NaN
1,595,6987,38,4Б,NaN,NaN,NaN,NaN,"поселок Ушково, Пляжевая улица, дом 4Б",True,False,NaN,128.0,"пос. Ушково, Пляжевая ул., д. 4Б",NaN,NaN
2,7134,6469,15,30,2,Е,NaN,NaN,"г.Санкт-Петербург, проспект Маршала Жукова, до...",True,False,Нежилое,30.0,"г.Санкт-Петербург, пр. Маршала Жукова, д. 30, ...",198303,NaN
3,124415,7838,38,5,2,А,NaN,NaN,"поселок Белоостров, Дюны, Центральная улица, д...",True,False,Нежилое,110.0,"пос. Белоостров, Дюны, Центральная ул., д. 5, ...",NaN,NaN
4,185368,4224,38,28,NaN,Б,NaN,NaN,"поселок Песочный, Речная улица, дом 28, литера Б",True,False,NaN,118.0,"пос. Песочный, Речная ул., д. 28, л. Б",NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166640,18941,5289,32,3,NaN,В,NaN,NaN,"г.Санкт-Петербург, Бородинская улица, дом 3, л...",True,True,Нежилое,4.0,"г.Санкт-Петербург, Бородинская ул., д. 3, л. В",190000.0,NaN
166641,18942,5031,35,34/3,NaN,А,NaN,NaN,"посёлок Александровская, 2-я линия, дом 34/3, ...",True,True,Жилое,109.0,"пос. Александровская, 2-я линия, д. 34/3, л. А",196631.0,NaN
166642,18943,5623,38,39,NaN,NaN,NaN,NaN,"посёлок Ушково, Детская улица, дом 39",True,False,NaN,128.0,"пос. Ушково, Детская ул., д. 39",NaN,NaN
166643,18944,5787,34,25,NaN,Б,NaN,NaN,"г.Санкт-Петербург, Заусадебная улица, дом 25, ...",True,False,Нежилое,68.0,"г.Санкт-Петербург, Заусадебная ул., д. 25, л. Б",NaN,NaN


In [71]:
df = pd.concat((df[['address', 'target_building_id']], df[['target_address', 'target_building_id']].rename(columns={'target_address': 'address'})))

In [68]:
text[:150]

'196641, Санкт-Петербург г, Школьная ул, д.5\nСанкт-Петербург (г.), Железнодорожный (пр-кт.), д.14 2 1\n198325, Санкт-Петербург г, Красное Село г, Театра'

In [46]:
import re
line = 'asdasfтерритория предприятия asda'
line = re.sub(
           str(areatype.loc[0, 'name']), 
           str(areatype.loc[0, 'name']), 
           line
       )

In [47]:
line

'asdasfgay asda'

In [264]:
area 
areatype
district
geonim
# geonimtype
# prefix
# subrf
# town

,id,type_id,name,short_name,is_updated,is_actual,only_name
0,100038,19,ЗСД,ЗСД,True,True,ЗСД
1,502,2,3-я аллея,3-я аллея,True,True,3-я
2,634,12,Новая улица,Новая ул.,True,True,Новая
3,710,8,4-й переулок,4-й пер.,True,True,4-й
4,771,8,Озерной переулок,Озерной пер.,True,True,Озерной
...,...,...,...,...,...,...,...
6202,5600,6,5-я линия,5-я линия,True,True,5-я
6203,6126,2,2-я аллея,2-я аллея,True,True,2-я
6204,100040,17,парк Приоратский,парк Приоратский,True,True,Приоратский
6205,100041,2,Главная,Главная,True,True,Главная


In [282]:
geonim = geonim[geonim['name'] != geonim['short_name']].reset_index(drop=True)

In [4]:
train = pd.read_csv('train.csv').drop('Unnamed: 0', axis=1)

In [8]:
from collections import Counter

In [285]:
train = train.reset_index(drop=True)

In [286]:
train

,target_building_id,address
0,210852.0,"г.Санкт-Петербург, Софийская ул., д. 8, к. 3, ..."
1,140073.0,"г.Санкт-Петербург, Мучной пер., д. 1/38, л. Б"
2,86677.0,"Фонтанки наб., 203БВ"
3,54442.0,"г.Санкт-Петербург, Масляный переулок, дом 8, л..."
4,84553.0,"Екатерининский парк, 9"
...,...,...
411815,140861.0,"г.Санкт-Петербург, Днепропетровская улица, д. ..."
411816,21664.0,"г.Санкт-Петербург, Днепропетровская улицаца, д..."
411817,97895.0,"г.Санкт-Петербург, Днепропетровская улицаца, д..."
411818,64389.0,"г.Санкт-Петербург, Днепропетровская улицаца, д..."


In [287]:
text = "\n".join(train['address'])
new_train = train.copy()

for i in tqdm(range(len(geonim))):   
#     new_text = re.sub(
#            str(geonimtype.loc[i, 'name']), 
#            str(geonimtype.loc[i, 'short_name']), 
#            text
#        )

#     if new_line != line:
#         new_df.loc[i, 'address'] = new_line
#         break

    new_text = re.sub(
           str(geonim.loc[i, 'short_name']), 
           str(geonim.loc[i, 'name']), 
           text
       )
    
    ndf = pd.DataFrame(new_text.split("\n")).rename(columns={0: 'address'})
    ndf['target_building_id'] = train['target_building_id']
    new_train = pd.concat((new_train, ndf)).drop_duplicates()
#     if new_line != line:
#         new_df.loc[i, 'address'] = new_line
#         break
        

 40%|███████████████████████████████▍                                              | 1850/4586 [34:22<50:50,  1.11s/it]


KeyboardInterrupt: 

In [290]:
new_train = new_train.reset_index(drop=True)

In [291]:
new_train = new_train.drop_duplicates()

In [292]:
train = new_train.copy()

In [293]:
train.to_csv('train.csv')

In [294]:
train

,target_building_id,address
0,210852.0,"г.Санкт-Петербург, Софийская ул., д. 8, к. 3, ..."
1,140073.0,"г.Санкт-Петербург, Мучной пер., д. 1/38, л. Б"
2,86677.0,"Фонтанки наб., 203БВ"
3,54442.0,"г.Санкт-Петербург, Масляный переулок, дом 8, л..."
4,84553.0,"Екатерининский парк, 9"
...,...,...
568640,11171.0,"г.Санкт-Петербург, проспект Косыгина, д. 13, л. А"
568641,202012.0,"г.Санкт-Петербург, проспект Косыгина"
568642,48212.0,"г.Санкт-Петербург, проспект Косыгина, д. 30, к..."
568643,71176.0,"г.Санкт-Петербург, проспект Косыгина, д. 2, к. 5"


In [213]:
ndf = pd.DataFrame(new_text.split("\n")).rename(columns={0: 'address'})
# ndf['target_building_id'] = df['target_building_id']

In [214]:
ndf

,address
0,"г.Санкт-Петербург, Софийская улица, д. 8, к. 3..."
1,"г.Санкт-Петербург, Мучной переулицак, д. 1/38,..."
2,"Фонтанки набережная, 203БВ"
3,"г.Санкт-Петербург, Масляный переулицакулицак, ..."
4,"Екатерининский парк, 9"
...,...
324230,"г.Санкт-Петербург, улицаца Ткачей, дом 26, лит..."
324231,"г.Санкт-Петербург, участок ж/д ""Балтийский вок..."
324232,"г.Санкт-Петербург, проспект Большевиков, д. 38..."
324233,"г.Санкт-Петербург, Новоутиная улица, д. 6, л. А"


In [204]:
ndf['target_building_id'] = train['target_building_id']

In [205]:
train.drop_duplicates()

,target_building_id,address
0,210852.0,"г.Санкт-Петербург, Софийская ул., д. 8, к. 3, ..."
1,140073.0,"г.Санкт-Петербург, Мучной пер., д. 1/38, л. Б"
2,86677.0,"Фонтанки наб., 203БВ"
3,54442.0,"г.Санкт-Петербург, Масляный переулок, дом 8, л..."
4,84553.0,"Екатерининский парк, 9"
...,...,...
324229,32412.0,"г.Санкт-Петербург, улица Крыленко, дом 39/29, ..."
324231,204893.0,"г.Санкт-Петербург, участок ж/д ""Балтийский вок..."
324232,82640.0,"г.Санкт-Петербург, пр. Большевиков, д. 38, к. ..."
324233,27920.0,"г.Санкт-Петербург, Новоутиная ул., д. 6, л. А"


In [206]:
pd.concat((train, ndf)).drop_duplicates()

,target_building_id,address
0,210852.0,"г.Санкт-Петербург, Софийская ул., д. 8, к. 3, ..."
1,140073.0,"г.Санкт-Петербург, Мучной пер., д. 1/38, л. Б"
2,86677.0,"Фонтанки наб., 203БВ"
3,54442.0,"г.Санкт-Петербург, Масляный переулок, дом 8, л..."
4,84553.0,"Екатерининский парк, 9"
...,...,...
324229,32412.0,"г.Санкт-Петербург, улица Крыленко, дом 39/29, ..."
324231,204893.0,"г.Санкт-Петербург, участок ж/д ""Балтийский вок..."
324232,82640.0,"г.Санкт-Петербург, пр. Большевиков, д. 38, к. ..."
324233,27920.0,"г.Санкт-Петербург, Новоутиная ул., д. 6, л. А"


In [38]:
areatype

,id,name,short_name,is_updated,is_actual
0,27,территория предприятия,территория предприятия,True,True
1,29,земли,земли,True,True
2,30,садоводство,сад-во,True,True
3,31,дачный посёлок,дачный посёлок,True,True
4,32,берег,берег,True,True
...,...,...,...,...,...
67,6340,канал,кан.,True,True
68,6468,автодорога,автодорога,True,True
69,6735,ВЛ,ВЛ,True,True
70,6831,сети,сети,True,True


In [36]:
df

,address,target_building_id
0,"196641, Санкт-Петербург г, Школьная ул, д.5",76337.0
1,"Санкт-Петербург (г.), Железнодорожный (пр-кт.)...",220011.0
2,"198325, Санкт-Петербург г, Красное Село г, Теа...",109979.0
3,"195256, Санкт-Петербург (г.), Верности (ул.), ...",72112.0
4,"194291, Санкт-Петербург (г.), Кустодиева (ул.)...",186531.0
...,...,...
138283,"муниципальный округ Константиновское, территор...",80244.0
138284,"г.Санкт-Петербург, Кирочная улица, дом 28, лит...",93144.0
138285,"г.Санкт-Петербург, Ленская улица, дом 6, корпу...",98201.0
138286,"посёлок Парголово, проспект Энгельса, дом 162,...",15911.0


In [ ]:
tokenizer = WordPunctTokenizer()
def tokenize(x):
    return ' '.join(tokenizer.tokenize(x.lower()))

with open('data.txt', 'w') as f:
    for line in df['adress']:
        f.write(tokenize(line) + '\n')

In [ ]:
# строим и применяем bpe кодирование
learn_bpe(open('data.txt', encoding='utf-8'), open('bpe_rules', 'w', encoding='utf-8'), num_symbols=8000)
bpe = BPE(open('./bpe_rules', encoding='utf-8'))

with open('train.bpe', 'w', encoding='utf-8') as f_out:
    for line in open('data.txt', encoding='utf-8'):
        f_out.write(bpe.process_line(line.strip()) + '\n')

In [ ]:
data = np.array(open('./train.bpe', encoding='utf-8').read().split('\n'))[:-1]

In [ ]:
data = data[:, np.newaxis]
target = df['target_building_id'].to_numpy()[:, np.newaxis]

In [ ]:
data = np.concatenate((data, target), axis=1)

In [ ]:
train_inp, dev_inp, train_target, dev_target = train_test_split(data[:, 0], data[:, 1].astype(int), test_size=3000,
                                                          random_state=42)

In [ ]:
inp_voc = Vocab.from_lines(train_inp)

In [ ]:
# тут можно посмотреть, как работает мапинг из индекса в токен и наоборот
batch_lines = sorted(train_inp, key=len)[5:10]
batch_ids = inp_voc.to_matrix(batch_lines)
batch_lines_restored = inp_voc.to_lines(batch_ids)

print("lines")
print(batch_lines)
print("\nwords to ids (0 = bos, 1 = eos):")
print(batch_ids)
print("\nback to words")
print(batch_lines_restored)

In [ ]:
class CPDataset(Dataset): 
    def __init__(self, inp, target):
        self.inp = inp
        self.target = target
        
    def __len__(self):
        return len(self.inp)
        
    def __getitem__(self, idx):
        return {'adress': self.inp[idx],
                'id': int(self.target[idx])}

In [ ]:
class BaseModel(nn.Module):
    def __init__(self, inp_voc, emb_size=64, hid_size=128):
        """
        Базовая модель encoder-decoder архитектуры
        """
        super().__init__() 

        self.inp_voc = inp_voc
        self.hid_size = hid_size
        
        self.emb_inp = nn.Embedding(len(inp_voc), emb_size)
        self.enc0 = nn.GRU(emb_size, hid_size, batch_first=True)

        self.fc = nn.Linear(hid_size, 3)
        
    def forward(self, inp):
        """ Сначала примените  encode а затем decode"""
        ans = self.encode(inp)
        return ans


    def encode(self, inp, **flags):
        """
        Считаем скрытое состояние, которое будет начальным для decode
        :param inp: матрица входных токенов
        :returns: скрытое представление с которого будет начинаться decode
        """
        inp_emb = self.emb_inp(inp)
        batch_size = inp.shape[0]
        
        enc_seq, [last_state_but_not_really] = self.enc0(inp_emb)
        # enc_seq: [batch, time, hid_size], last_state: [batch, hid_size]
        
        # последний токен, не последние на самом деле, так как мы делали pading, чтобы тексты были
        # одинакового размер, поэтому подсчитать длину исходного предложения не так уж тривиально
        lengths = (inp != self.inp_voc.eos_ix).to(torch.int64).sum(dim=1).clamp_max(inp.shape[1] - 1)
        last_state = enc_seq[torch.arange(len(enc_seq)), lengths]
        # ^-- shape: [batch_size, hid_size]
        
#         out = self.fc(last_state)
        return last_state

In [ ]:
class Fitter(object):
    def __init__(
        self,
        model,
        opt,
        loss,
        train_dataloader,
        scheduler,
        dev_dataloader,
        batch_size = 32,
        sheduler_steps = 10,
        n_epochs = 10
    ):

        self.model = model
        self.n_epochs = n_epochs
        self.sheduler_steps = sheduler_steps

        self.opt = opt
        self.loss = loss
        self.scheduler = scheduler
        self.train_dataloader = train_dataloader
        self.dev_dataloader = dev_dataloader
        
        self.model.to(device)
        self.metrics = {'train_loss': [], 'dev_ROC-AUC': []}

    def fit(self):
        # Turn on training
        self.model.train(True)
        
        # Fit
        for epoch in range(self.n_epochs):
            train_loss = 0
            for i, batch in tqdm(enumerate(
                self.train_dataloader, desc='Работаем Братья', total=len(self.train_dataloader) // self.batch_size):
                
                targets = torch.tensor(batch['id'], device=device)
                self.opt.zero_grad()
                embeddings = self.model(batch['adress'].to(device))
                loss = self.loss(embeddings, targets)
                loss.backward()
                self.opt.step()
                train_loss += loss.item()
                                 
            
            if epoch % self.sheduler_steps == 1:
                self.scheduler.step()
            
            clear_output(wait=True)  
            self.model.eval()
            with torch.no_grad():
                self.metrics['train_loss'] += [train_loss / (i+1)]
                preds = F.softmax(self.model(self.model.inp_voc.to_matrix(self.dev[0]).to(device)), dim=1).detach().cpu().numpy()
                self.metrics['dev_ROC-AUC'] += [roc_auc_score(self.dev[1],
                                                              preds,
                                                              multi_class='ovr')]
                
            if len(self.metrics['dev_ROC-AUC']) > 2 and self.metrics['dev_ROC-AUC'][-1] > self.metrics['dev_ROC-AUC'][-2]:
                torch.save(self.model, 'rnn_segm.pt')
            
            plt.figure(figsize=(10, 5))
            
            plt.subplot(1, 2, 1)
            plt.xlabel("Iteration")
            plt.ylabel("Train Loss")
            plt.grid()
            plt.plot(self.metrics['train_loss'])

            plt.subplot(1, 2, 2)
            plt.xlabel("Iteration")
            plt.ylabel("ROC-AUC")
            plt.plot(self.metrics['dev_ROC-AUC'], color="orange")
            plt.grid()
            
            plt.show()
            
            self.model.train(True)
            
        # Turn off training
        self.model.train(False)

In [ ]:
model = BaseModel(inp_voc).to(device)
opt = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = ExponentialLR(opt, gamma=0.9)
train_dataset = CPDataset(train_inp, train_target)
criterion = TripletLossWithMiner(margin=0.1, miner=HardTripletsMiner(), need_logs=True)
sampler = BalanceSampler(train_dataset.get_labels(), n_labels=4, n_instances=5)
train_loader = DataLoader(train_dataset, batch_sampler=sampler)
dev_loader = DataLoader(train_dataset, batch_sampler=sampler)

sheduler_steps = 1

epochs = 15

fitter =  Fitter(
                model,
                opt,
                criterion,
                train_loader,
                scheduler,
                dev_loader,
                sheduler_steps = sheduler_steps,
                n_epochs = epochs,
)